# Give clearer instructions






### 1:Few-shot learning
A common way to adapt language models to new tasks is to use few-shot learning. In few-shot learning, a set of training examples is provided as part of the prompt to give additional context to the model.

When using the Chat Completions API, a series of messages between the User and Assistant (written in the new prompt format), can serve as examples for few-shot learning. These examples can be used to prime the model to respond in a certain way, emulate particular behaviors, and seed answers to common questions.

In [ ]:

#r "nuget: Azure.AI.OpenAI, 1.0.0-beta.5"

using Azure;
using Azure.AI.OpenAI;
using static System.Environment;

// Import the helper class to load configuration values
#!import ../Config/TheHelper.cs

// Load configuration values
string endpoint = TheHelper.GetConfigurationValue("endpoint");
string apiKey = TheHelper.GetConfigurationValue("apikey");
string deploymentName = TheHelper.GetConfigurationValue("DeploymentName");

// Create the OpenAI client
OpenAIClient client = new(new Uri(endpoint), new AzureKeyCredential(apiKey));

// Print a message to indicate that the client is ready to use
Console.WriteLine("OpenAI client is ready to use.");

**System Message**

Assistant is an intelligent chatbot designed to help users answer their tax related questions.
Instructions
- Only answer questions related to taxes.
- If you're unsure of an answer, you can say "I don't know" or "I'm not sure" and recommend users go to the IRS website for more information.

**Few-shot examples**
- User - "When do I need to file my taxes by?"
- Assistant - "In 2023, you will need to file your taxes by April 18th. The date falls after the usual April 15th deadline because April 15th falls on a Saturday in 2023. For more details, see https://www.irs.gov/filing/individuals/when-to-file "

- User - "How can I check the status of my tax refund?"
- Assistant - "You can check the status of your tax refund by visiting https://www.irs.gov/refunds "

In [ ]:
// Create a new ChatCompletionsOptions object
var chatCompletionsOptions = new ChatCompletionsOptions()
{
    // Define the messages to be used in the chat
    Messages =
    {
        // Add a system message to explain the purpose of the chatbot
        new ChatMessage(ChatRole.System, 
        @"Assistant is an intelligent chatbot designed to help users answer their tax related questions.
        Instructions
        - Only answer questions related to taxes.
        - If you're unsure of an answer, you can say ""I don't know"" or ""I'm not sure"" and recommend users go to the IRS website for more information." ),
        
        // Add a user message to ask when taxes need to be filed
        new ChatMessage(ChatRole.User, "When do I need to file my taxes by?"),
        
        // Add an assistant message to answer the user's question
        new ChatMessage(ChatRole.Assistant, @"In 2023, you will need to file your taxes by April 18th. The date falls after the usual April 15th deadline 
        because April 15th falls on a Saturday in 2023. For more details, see https://www.irs.gov/filing/individuals/when-to-file"),
        
        // Add a user message to ask how to check the status of a tax refund
        new ChatMessage(ChatRole.User, "How can I check the status of my tax refund?"),
        
        // Add an assistant message to answer the user's question
        new ChatMessage(ChatRole.Assistant, "You can check the status of your tax refund by visiting https://www.irs.gov/refunds"),
    },
    
    // Set the maximum number of tokens to be used in the chat
    MaxTokens = 100
};

// Print a message to indicate that the chat messages are ready to use
Console.WriteLine("System Message and fewshots ready.");

In this code, we are testing a question that falls `outside the scope` of the chatbot.

In [ ]:
// Add a user message to ask a non-tax related question
chatCompletionsOptions.Messages.Add(new ChatMessage(ChatRole.User, "When was the last Red Sox Game?"));

// Get chat completions from the OpenAI API using the specified deployment name and chat options
Response<ChatCompletions> response = client.GetChatCompletions(deploymentOrModelName: deploymentName, chatCompletionsOptions);

// Print the response message content to the console
Console.WriteLine(TheHelper.ConsoleWriteLine(response.Value.Choices[0].Message.Content));

// Print a blank line to separate the output from the next message
Console.WriteLine();

Now, let ask for something realted to Taxes

In [ ]:
// Add a user message to ask a tax-related question
chatCompletionsOptions.Messages.Add(new ChatMessage(ChatRole.User, "Is a contribution to the Boston Fine Arts Museum tax deductible?"));

// Get chat completions from the OpenAI API using the specified deployment name and chat options
Response<ChatCompletions> response = client.GetChatCompletions(deploymentOrModelName: deploymentName, chatCompletionsOptions);

// Print the response message content to the console
Console.WriteLine(TheHelper.ConsoleWriteLine(response.Value.Choices[0].Message.Content));

// Print a blank line to separate the output from the next message
Console.WriteLine();

### 2: Ask for a structured output
The outcome format plays a crucial role in system-to-system integration, especially when an application is making a call to 'copilot' and receiving the response in JSON format. This format greatly facilitates parsing and is highly advantageous in terms of ease of use.


In [ ]:
// Add a user message to ask for a list of deductible expenses in JSON format
chatCompletionsOptions.Messages.Add(new ChatMessage(ChatRole.User, "Generate a list of 2 deductible text expenses, in json format document, with the following keys: Expense, description"));

// Get chat completions from the OpenAI API using the specified deployment name and chat options
Response<ChatCompletions> response = client.GetChatCompletions(deploymentOrModelName: deploymentName, chatCompletionsOptions);

// Print the response message content to the console
Console.WriteLine(response.Value.Choices[0].Message.Content);

// Print a blank line to separate the output from the next message
Console.WriteLine();

### 3: Ask the model to check whether conditions are satisfied
Instruct the model to verify conditions. This prevents irrelevant or incorrect responses. For example, ask the model to rewrite instructions in a specific format or indicate `"No instruction steps provided"` if no instructions are found. The model follows these instructions accurately based on the conditions being satisfied or not.

A. Extract instructions from a text.

In [ ]:
// Define the text to be analyzed
//This description about how to fill taxes in US was generated by CHATGPT using Shakespeare tone.
string Shakespeare = @"Good friends, attend to these words most wise, For I shall share with thee five steps to complete thy tax declaration with no surprise.
Gather all documents that show thy income and expenses, For these shall be the basis of thy tax declaration hence.
Determine if thou art eligible for deductions or credits most fair, For these can reduce thy tax burden with much care.
Calculate thy adjusted gross income with precision, For this shall be the basis of thy tax calculation.
Choose whether to itemize or take the standard deduction with thought, For this decision can affect how much tax thou ought.
And finally, complete thy tax forms with care and attention, For accuracy is key to avoid any contention.
Thus have I shared with thee five steps to complete thy tax declaration with ease, May it bring thee peace of mind and much financial ease.";

// Prompt the user to analyze the text and rewrite the instructions as a list
string CheckConditionPrompt = @"Analyse the following text and if it contains sequence of instructions, re-write those instructions as a list. 
If you don't find any sntriction say: ""No instruction steps provided, this is the way."" This is the Text: " + Shakespeare;

// Send the prompt to the chatbot and get a response
chatCompletionsOptions.Messages.Add(new ChatMessage(ChatRole.User, CheckConditionPrompt));
Response<ChatCompletions> response = client.GetChatCompletions(deploymentOrModelName: deploymentName, chatCompletionsOptions);

// Print the response to the console
Console.WriteLine(response.Value.Choices[0].Message.Content);
Console.WriteLine();

B. Try a text without instructions on it.

In [ ]:
//Henry Wadsworth Longfellow, and the poem is titled "A Psalm of Life."
string Longfellow=@"Tell me not, in mournful numbers,
""Life is but an empty dream!""
For the soul is dead that slumbers,
And things are not what they seem.

Life is real! Life is earnest!
And the grave is not its goal;
""Dust thou art, to dust returnest,""
Was not spoken of the soul.";

string CheckConditionPrompt = @"Analyse the following text and if it contains sequence of instructions, re-write those instructions as a list. 
If you don't find any instructions say: ""No instruction steps provided, this is the way."". This is the Text: " + Longfellow;

// Send the prompt to the chatbot and get a response
chatCompletionsOptions.Messages.Add(new ChatMessage(ChatRole.User, CheckConditionPrompt));
Response<ChatCompletions> response = client.GetChatCompletions(deploymentOrModelName: deploymentName, chatCompletionsOptions);

// Print the response to the console
Console.WriteLine(response.Value.Choices[0].Message.Content);
Console.WriteLine("Longfellow");

Next Sample [Split complex task into simpler sub task](./3.SplitComplexTaskIntoSimplerSubTask.ipynb)